<a href="https://colab.research.google.com/github/a00700c/AI-teamproject/blob/main/%ED%98%84%EC%8A%B9%EA%B5%AC/insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd

In [86]:
df = pd.read_csv("/content/sample_data/insurance.csv")

In [87]:
df.head()
#CRIM 근방 범죄율
#ZN 주택지 비율
#INDUS 상업적 비지니스에 활용되지 않는 농지 면적
#CHAS 경계선에 강에 있는지 여부
#NOX 산화 질소 농도
#RM 자택당 평균 방 갯수
#AGE 1940 년 이전에 건설된 비율
#DIS 5 개의 보스턴 고용 센터와의 거리에 다른 가주이 부여
#RAD radial 고속도로와의 접근성 지수
#TAX 10000달러당 재산세
#PTRATIO 지역별 학생-교사 비율
#B 지역의 흑인 지수
#LSTAT 빈공층의 비율

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [88]:
df.shape
from sklearn.preprocessing import LabelEncoder
#sex
le = LabelEncoder()
le.fit(df.sex.drop_duplicates()) 
df.sex = le.transform(df.sex)
# smoker or not
le.fit(df.smoker.drop_duplicates()) 
df.smoker = le.transform(df.smoker)
#region
le.fit(df.region.drop_duplicates()) 
df.region = le.transform(df.region)

In [89]:
target = df['charges']
data = df.drop(['charges'], axis = 1)

In [90]:
target.head()

0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64

In [91]:
data.head()

,age,sex,bmi,children,smoker,region
0,19,0,27.900,0,1,3
1,18,1,33.770,1,0,2
2,28,1,33.000,3,0,2
3,33,1,22.705,0,0,1
4,32,1,28.880,0,0,1


In [92]:
#XBGRegressor 회귀 모델 사용하기 위한 import
from xgboost import XGBRegressor 

In [93]:
# 데이터 plot 과 어떤 feature가 가중치가 높은지 확인하기 위한 라이브러리 import
from xgboost import plot_importance, plot_tree
import graphviz
import matplotlib.pyplot as plt
import pandas as pd

In [94]:
# 모델의 하이퍼 파라미터 설정
# 의사결정 트리수 (n_estimators) 100개로 정의
# 사용할 코어수 (n_jobs) 최대치로 사용
# learing_rate 초기값 0.01 사용
# 모델 여러개 돌리기 위해 사이킷런의 그리드 서치 사용
from sklearn.model_selection import GridSearchCV, train_test_split
xgb = XGBRegressor()

In [95]:
#하이퍼 파라미터 종류
# learning_rate // 학습률
# n_estimators // 학습기의 갯수(반복 수행 횟수)
# min_child_weight leaf와 유사, 과적합 조절용
# max_depth 트리의 최대 깊이
# subsample 샘플링하는 비율
# early_stopping_rounds 더 이상 비용 평가 지표가 감소하지 않는 최대 반복 횟수
# eval_metric 반복 수행 시 사용하는 비용 평가 지표
# eval_set 평가를 수행하는 별도의 검증 데이터 세트, 일반적으로 검증 세트에서 반복적으로 비용 감소 성능 평가
params = {'n_estimators': [i * 50 for i in range(10,15)], 
          'learning_rate': [i * 0.01 for i in range(0, 30, 3)], 
          'n_jobs': [-1], 
          'max_depth': [i * 4 for i in range(4)]}

In [96]:
x_train, t_test, x_valid, t_valid = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=7)

In [97]:
x_train, x_valid

(      age  sex     bmi  children  smoker  region
 1317   18    1  53.130         0       0       2
 794    39    0  31.920         2       0       1
 680    21    0  17.400         1       0       3
 795    27    1  28.500         0       1       1
 887    36    0  30.020         0       0       1
 ...   ...  ...     ...       ...     ...     ...
 211    40    1  30.875         4       0       1
 502    51    1  23.210         1       1       2
 537    46    0  30.200         2       0       3
 1220   30    0  21.945         1       0       0
 175    63    0  37.700         0       1       3
 
 [1070 rows x 6 columns], 1317     1163.46270
 794      7209.49180
 680      2585.26900
 795     18310.74200
 887      5272.17580
            ...     
 211      8162.71625
 502     22218.11490
 537      8825.08600
 1220     4718.20355
 175     48824.45000
 Name: charges, Length: 1070, dtype: float64)

In [98]:
gs = GridSearchCV(xgb, params, scoring='neg_mean_absolute_percentage_error', cv=2)

In [99]:
gs.fit(x_train, x_valid)

[15:36:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

GridSearchCV(cv=2, estimator=XGBRegressor(),
             param_grid={'learning_rate': [0.0, 0.03, 0.06, 0.09, 0.12, 0.15,
                                           0.18, 0.21, 0.24, 0.27],
                         'max_depth': [0, 4, 8, 12],
                         'n_estimators': [500, 550, 600, 650, 700],
                         'n_jobs': [-1]},
             scoring='neg_mean_absolute_percentage_error')

In [100]:
cv_result_df = pd.DataFrame(gs.cv_results_)
cv_result_df.sort_values(by=['rank_test_score'], inplace=True)

In [101]:
cv_result_df[['params', 'mean_test_score', 'rank_test_score']].head(100)

,params,mean_test_score,rank_test_score
25,"{'learning_rate': 0.03, 'max_depth': 4, 'n_est...",-0.355809,1
26,"{'learning_rate': 0.03, 'max_depth': 4, 'n_est...",-0.359811,2
27,"{'learning_rate': 0.03, 'max_depth': 4, 'n_est...",-0.363743,3
28,"{'learning_rate': 0.03, 'max_depth': 4, 'n_est...",-0.366885,4
195,"{'learning_rate': 0.27, 'max_depth': 12, 'n_es...",-0.367336,5
...,...,...,...
192,"{'learning_rate': 0.27, 'max_depth': 8, 'n_est...",-0.405829,96
193,"{'learning_rate': 0.27, 'max_depth': 8, 'n_est...",-0.405829,97
194,"{'learning_rate': 0.27, 'max_depth': 8, 'n_est...",-0.405829,98
48,"{'learning_rate': 0.06, 'max_depth': 4, 'n_est...",-0.409291,99


In [102]:
best = gs.best_estimator_

In [103]:
pred = best.predict(t_test)

In [104]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error
print('RMSE : ', mean_squared_error(pred, t_valid))
print('MAPE : ', mean_absolute_percentage_error(t_valid, pred))

RMSE :  23141419.810198482
MAPE :  0.3292038216093272


In [105]:
pred

array([11503.657 , 10441.775 ,  1075.1292, 17542.533 , 12054.073 ,
        2218.91  , 17151.357 ,  8922.885 ,  4535.1865, 10081.081 ,
        5266.692 ,  5686.5405,  5209.4546,  5079.788 , 12378.572 ,
       10871.582 ,  4242.347 , 13591.501 , 12438.426 , 12217.451 ,
        5525.9375,  4745.864 , 38478.812 , 25061.592 , 18224.951 ,
       21339.021 , 45986.227 , 38515.684 ,  7628.333 ,  7468.4185,
       12352.266 , 21898.803 ,  5620.5186, 14881.516 , 12885.906 ,
        2742.0337,  6352.733 ,  6095.892 ,  9075.873 ,  1747.863 ,
        9254.633 ,  5313.4385,  9662.772 ,  6340.9087,  5951.868 ,
        5955.444 ,  6017.2935, 45449.266 , 23204.484 , 14121.034 ,
        7927.5   , 21577.01  ,  9890.9   ,  9230.29  ,  2588.2422,
       42215.73  , 28386.2   , 11957.761 ,  4293.12  ,  3471.5337,
       10734.272 , 14728.377 ,  9813.285 , 17745.695 ,  7233.026 ,
       32657.676 , 14403.999 ,  2946.423 , 47689.523 ,  3507.2688,
       24730.615 ,  5880.9443,  5165.8657, 46320.766 ,  5508.5

In [106]:
result = pd.DataFrame(pred)

In [107]:
result.to_excel('결과.xlsx')
t_valid.to_excel('테스트대상.xlsx')

In [108]:
print('정확도', r2_score(pred, t_valid))

정확도 0.8319249431293481
